# Table of Contents
 <p><div class="lev1"><a href="#Warsztaty-metod-obliczeniowych-ab-initio">Warsztaty metod obliczeniowych <em>ab-initio</em></a></div><div class="lev2"><a href="#Wstęp">Wstęp</a></div><div class="lev2"><a href="#Plan">Plan</a></div>

# Warsztaty metod obliczeniowych *ab-initio*

**Paweł T. Jochym**

Zakład komputerowych badań materiałów

Instytut Fizyki Jądrowej PAN, Kraków

## Wstęp

Materiały pomocnicze do warsztatów mają formę zbioru zeszytów [Jupyter](https://jupyter.org/) dostępnych w publicznym repozytorium:
[https://github.com/jochym/abinitio-workshop](https://github.com/jochym/abinitio-workshop). 

Zeszyty można oglądnąć i eksperymentować na nich otwierając je w serwisie binder. Jedynym wymaganiem jest posiadanie działającej, współczesnej przeglądarki WWW (np. firefox,chrome,safari). Wystarczy kliknąć poniższą ikonę.

[![Binder](http://mybinder.org/badge.svg)](http://mybinder.org/repo/jochym/abinitio-workshop) 

*Wszystkie przykłady oraz konieczne dane a także pełny tekst prezentacji znajduje się w repozytorium.*

## Plan

* Schemat obliczeń
* Środowisko
* Wyliczenie struktury kryształu
* Własności elastyczne kryształu
* Dynamika molekularna nanocząstki
* Termodynamika nanocząstki